In [ ]:
import gensim
import pymystem3
import pandas as pd

In [ ]:
code = pd.read_csv('tmp/criminal_code.csv')
df = pd.read_csv('tmp/judicial_orders.csv').rename(columns={'title':'name'})

In [3]:
code['lemmas'] = code['lemmas'].map(eval)
df['lemmas'] = df['lemmas'].map(eval)
code['source'] = 'criminal_code'
df['source'] = 'court_orders'
df['body'] = df['intro'] + df['case'] + df['result']
df = pd.concat([code[['name', 'number', 'lemmas','body', 'source']], 
                df[['name', 'lemmas', 'body', 'source']]], ignore_index=True)

/home/gzhigalov/envs/diploma/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [4]:
td = lambda row: gensim.models.doc2vec.TaggedDocument(words=row['lemmas'], tags=[row.name])
documents = df.apply(td, axis=1).tolist()

In [ ]:
%%time
model = gensim.models.doc2vec.Doc2Vec()
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=3000)

In [ ]:
model.save('tmp/doc2vec/code_and_court_orders.model')

In [10]:
model = gensim.models.doc2vec.Doc2Vec.load('tmp/doc2vec/code_and_court_orders.model')

In [11]:
stem = pymystem3.Mystem(entire_input=False)

In [12]:
text = """Группой из 10 человек было совершено нападение на сберегательную кассу в момент её закрытия"""

In [13]:
new_vector = model.infer_vector(stem.lemmatize(text))
sims = model.docvecs.most_similar([new_vector], topn=100)

/home/gzhigalov/envs/diploma/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [14]:
import sqlite3
conn = sqlite3.connect('data.db')
df[df['source'] == 'criminal_code'].reset_index()[['index', 'name', 'number', 'body']].to_sql('data', con=conn, index=False, if_exists='replace')


In [15]:
def get_code_info(sims, n=5):
    cursor = conn.execute(f"SELECT * FROM data WHERE `index` in ({','.join([str(x[0]) for x in sims])})")
    data = cursor.fetchmany(n)
    cursor.close()
    return data

In [16]:
for i, p in sims:
    if df.loc[i, 'source'] == 'criminal_code':
        print(p, df.loc[i, 'name'])

0.4224820137023926 Общие условия уголовной ответственности
